In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import gandula
import copy

In [54]:
key = 'f6a8469524424b2dd4cacdcde6eee99603dbfc492df145ce2e281325e8ad473a'
events = gandula.get_match_events(4436)
events

[PFF_Event(4427560) [PFF_PossessionEventType.PASS=4245025],
 PFF_Event(4427697) [PFF_PossessionEventType.PASS=4245173],
 PFF_Event(4427699) [PFF_PossessionEventType.PASS=4245175],
 PFF_Event(4427701) [PFF_PossessionEventType.PASS=4245177],
 PFF_Event(4427705) [PFF_PossessionEventType.PASS=4245181],
 PFF_Event(4427706) [PFF_PossessionEventType.PASS=4245182],
 PFF_Event(4427708) [],
 PFF_Event(4427712) [PFF_PossessionEventType.PASS=4245186],
 PFF_Event(4427714) [PFF_PossessionEventType.CHALLENGE=4245189, PFF_PossessionEventType.PASS=4245190],
 PFF_Event(4427715) [],
 PFF_Event(4427716) [PFF_PossessionEventType.CHALLENGE=4245192],
 PFF_Event(4427718) [],
 PFF_Event(4427719) [PFF_PossessionEventType.PASS=4245193],
 PFF_Event(4427721) [PFF_PossessionEventType.PASS=4245196],
 PFF_Event(4427723) [PFF_PossessionEventType.PASS=4245197],
 PFF_Event(4427727) [PFF_PossessionEventType.PASS=4245202],
 PFF_Event(4427730) [PFF_PossessionEventType.PASS=4245204],
 PFF_Event(4427733) [PFF_PossessionEvent

In [72]:
events[0].possessionEvents[0].passingEvent.passerPlayer.id

'1940'

In [94]:
events[10].game.id

AttributeError: 'NoneType' object has no attribute 'id'

In [107]:
events[0].possessionEvents[0].game.id

'4436'

In [111]:
import pandas as pd

def events_to_df(events, match_id):
    """
    Converte uma lista de 'events' em um DataFrame com colunas:
      match_id, event_id, possession_id, possession_type, 
      player_id, receiver, outcome, carry_type
    """

    rows = []

    for event in events:
        # Aqui assumimos que existe 'event.id' e 'event.game.id' (ou algo similar),
        # para identificar o match_id e event_id. Ajuste conforme seu modelo real.
        event_id = event.id if event.id else None
        team_id = event.team.id if event.team else None

        # Itera sobre cada posse (possessionEvent) dentro de event
        for possessionEvent in event.possessionEvents:
            possession_id = possessionEvent.id

            # Se for um PASS
            if possessionEvent.passingEvent:
                pass_event = possessionEvent.passingEvent
                passer = pass_event.passerPlayer.id if pass_event.passerPlayer else None
                receiver = pass_event.receiverPlayer.id if pass_event.receiverPlayer else None
                outcome = pass_event.passOutcomeType.value if pass_event.passOutcomeType else None
                

                row = {
                    "match_id": match_id,
                    "team_id": team_id,
                    "event_id": event_id,
                    "possession_id": possession_id,
                    "possession_type": "PASS",
                    "player_id": passer,
                    "receiver": receiver,
                    "outcome": outcome,
                    "carry_type": None
                }
                rows.append(row)

            # Se for um CARRY
            elif possessionEvent.ballCarryEvent:
                carry_event = possessionEvent.ballCarryEvent
                carrier = carry_event.ballCarrierPlayer.id if carry_event.ballCarrierPlayer else None
                dribble_outcome = carry_event.dribbleOutcomeType.value if carry_event.dribbleOutcomeType else None
                carry_type = carry_event.ballCarryType.value if carry_event.ballCarryType else None

                row = {
                    "match_id": match_id,
                    "event_id": event_id,
                    "possession_id": possession_id,
                    "possession_type": "CARRY",
                    "player_id": carrier,
                    "receiver": None,
                    "outcome": dribble_outcome,
                    "carry_type": carry_type
                }
                rows.append(row)

            # Se quiser ignorar outros tipos de eventos, basta não fazer nada

    # Converte a lista de dicionários em um DataFrame
    df = pd.DataFrame(rows, columns=[
        "match_id",
        "event_id",
        "possession_id",
        "possession_type",
        "player_id",
        "receiver",
        "outcome",
        "carry_type"
    ])

    return df

In [113]:
events_to_df(events, 4436)

,match_id,event_id,possession_id,possession_type,player_id,receiver,outcome,carry_type
0,4436,4427560,4245025,PASS,1940,2073,C,None
1,4436,4427697,4245173,PASS,2073,5090,C,None
2,4436,4427699,4245175,PASS,5090,4213,C,None
3,4436,4427701,4245177,PASS,4213,2073,C,None
4,4436,4427705,4245181,PASS,2073,5090,C,None
...,...,...,...,...,...,...,...,...
1047,4436,4433363,4250632,CARRY,480,None,None,T
1048,4436,4433371,4250657,PASS,162,None,O,None
1049,4436,4433501,4250818,PASS,5090,2073,C,None
1050,4436,4433530,4250847,PASS,1380,None,D,None
